# 🚧 Building a database for crime reports
In this guided project from Dataquest, I'm going to create a simple database from the boston.csv file that contains information about crimes in boston.

Skills learned via this project:
- Creating a postgres database and schemas
- Creating a table and insertion of values from csv into it
- Creating users and granting access

## Database and Schema creation

In [1]:
import psycopg2
conn = psycopg2.connect(dbname="postgres", user="postgres")
conn.autocommit = True
cursor = conn.cursor()
cursor.execute("DROP DATABASE IF EXISTS crime_db;") # To avoid errors in case running the script more times
cursor.execute("CREATE DATABASE crime_db;")
conn.close()

In [2]:
conn = psycopg2.connect(dbname="crime_db", user="postgres")
conn.autocommit = True
cursor = conn.cursor()
cursor.execute("CREATE SCHEMA crimes;")
conn.close()

# Table creation and insertion of values
Before creating our table we need to familiarize ourselves with the data that the boston.csv holds. This is important so that we can set proper column names and data types for our table.

First let's print the header and first row to get a glimpse of the data structure.

In [3]:
import csv
with open("boston.csv") as file:
    reader = csv.reader(file)
    data = list(reader)
    header = data[0]
    first_row = data[1]
    print(header)
    print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


Now we're going to create a function that will allow us to do a simple exploration of each column.

In [4]:
# This function takes in a name of a csv file to be analysed and the index number of one of its columns
# Function prints and returns a quick summary in a form of dictionary that should help determine column
# types and sizes for future tables.
# These values can be accessed by using following keys: "Unique values", "No. of unique values", "Data types",
# First item, Longest item

def analyze_column(csv_filename, col_index):
    import yaml
    empty_set = set()
    type_set = set()
    longest_item = 0
    with open(csv_filename) as file:
        reader = csv.reader(file)
        data = list(reader)
    for row in data[1:]: # Avoids header so that we analyze the values only
        value = row[col_index]
        empty_set.add(value) # gets all unique values in the column
        type_set.add(type(yaml.safe_load(value))) # gets all unique data types in the column
        if len(value) > longest_item:
            longest_item = len(value)
    no_unique_values = len(empty_set)
    if no_unique_values < 10:
        print("Column ", data[0][col_index], " contains following values: ", empty_set)
    print("Column ", data[0][col_index], " contains ", no_unique_values, " values.")
    print("Data types present in this column are: ", type_set)
    print("First value in this column is: ", data[1][col_index])
    print("Max length of a value ", longest_item, '\n')
    return {
        "Unique values" : empty_set,
        "No. of unique values" : no_unique_values,
        "Data types" : type_set,
        "First item" : data[1][col_index],
        "Longest item" : longest_item
    }

In [5]:
for column in range(len(header)):
    analyze_column("boston.csv", column)

print("This dataset has: ",  len(data[1:]))

Column  incident_number  contains  298329  values.
Data types present in this column are:  {<class 'int'>}
First value in this column is:  1
Max length of a value  6 

Column  offense_code  contains  219  values.
Data types present in this column are:  {<class 'int'>}
First value in this column is:  619
Max length of a value  4 

Column  description  contains  239  values.
Data types present in this column are:  {<class 'str'>}
First value in this column is:  LARCENY ALL OTHERS
Max length of a value  58 

Column  date  contains  1177  values.
Data types present in this column are:  {<class 'datetime.date'>}
First value in this column is:  2018-09-02
Max length of a value  10 

Column  day_of_the_week  contains following values:  {'Friday', 'Saturday', 'Sunday', 'Thursday', 'Monday', 'Tuesday', 'Wednesday'}
Column  day_of_the_week  contains  7  values.
Data types present in this column are:  {<class 'str'>}
First value in this column is:  Sunday
Max length of a value  9 

Column  lat  c

### Determination of data types

Every table needs to have a primary key column, a column that only hold unique values and serves as a unique identifier of each row of our database. So we are looking for a column that has the same number of unique values as is the number of non-header rows in our csv file. 

Based on the output of our function, we can see that the incident number column appears to be a good candidate as it is the only column that has 298 329, which is also the number of rows of our dataset.

For the remainder of our columns we will use the information about the datatype and the length of the longest value in the column. (The values in a csv files are originally all stored as strings and therefore for ints and floats we will use the lnegth in conjoncution with the data type to define what type of numeric data type we need. Note a "-" sign will be counted towards the length.)
- offense code is an integer of maximal length 4 therefore we only need a **smallint** (range: -32768 to +32767)
- description is a string with maximal length 58 chars but to account for larger descriptions we'll use **varchar(100)**
- incident_date is a date type without time information therefore we will use **date**
- day of the week contains only 7 values therefore we'll create our own enumeration type **week_days**
- latitude and longitude contain floats with 8 numbers after the decimal point therefore we will use **float8**

In [6]:
conn = psycopg2.connect(dbname="crime_db", user="postgres")
cursor = conn.cursor()
conn.autocommit = True
cursor.execute("""
    CREATE TYPE week_days AS ENUM (
    'Monday','Sunday', 'Tuesday', 'Friday', 'Wednesday', 'Saturday', 'Thursday');
""")
cursor.execute("""
CREATE TABLE crimes.boston_crimes (
    incident_id int PRIMARY KEY,
    offense_code smallint,
    description varchar(100),
    incident_date date,
    day_of_the_week week_days,
    latitude float8,
    longitude float8)
    """)
conn.close()

### Inserting rows into our table
For the data insertion we will use the copy_expert method that enables us to copy all contents of a csv into the rows of our table one by one. We will use the header option to skip the first row in the csv.

In [7]:
conn = psycopg2.connect(dbname="crime_db", user="postgres")
cursor = conn.cursor()
with open("boston.csv") as f:
    cursor.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", f)
conn.commit()

In [8]:
cursor.execute("SELECT * FROM crimes.boston_crimes LIMIT 5")

print(cursor.fetchall())

[(1, 619, 'LARCENY ALL OTHERS', datetime.date(2018, 9, 2), 'Sunday', 42.35779134, -71.13937053), (2, 1402, 'VANDALISM', datetime.date(2018, 8, 21), 'Tuesday', 42.30682138, -71.06030035), (3, 3410, 'TOWED MOTOR VEHICLE', datetime.date(2018, 9, 3), 'Monday', 42.34658879, -71.07242943), (4, 3114, 'INVESTIGATE PROPERTY', datetime.date(2018, 9, 3), 'Monday', 42.33418175, -71.07866441), (5, 3114, 'INVESTIGATE PROPERTY', datetime.date(2018, 9, 3), 'Monday', 42.27536542, -71.09036101)]


## Creating users and granting access
For security reasons it is recommended to take the approach of least privilege for all users of our database. We don't have real users in this database but we will simulate a situation where we need two user groups, one that can only ready what is in the database and the second one that can also modify tables. Though the right to create and delete table sits only with the superuser for the moment. This is usually the owner and administrator of the database.

First we need to prevent that any user/group ingerits any privilege from the public group. Therefore we will revoke all of its privileges on both the database and the schema.

In [9]:
cursor.execute("REVOKE ALL ON SCHEMA public FROM public;")
cursor.execute("REVOKE ALL ON DATABASE crime_db FROM public;")
conn.commit()

Now, let's create the two user groups and assign them the rights as outlined in the beginning of this section. Because we also need the users to be able to connect to the database we need to grand CONNECT privileges as well.

In [10]:
cursor.execute("DROP GROUP IF EXISTS readonly;")
cursor.execute("DROP GROUP IF EXISTS readwrite ;")

cursor.execute("CREATE GROUP readonly NOLOGIN;")
cursor.execute("CREATE GROUP readwrite NOLOGIN;")

cursor.execute("GRANT CONNECT ON DATABASE crime_db TO readonly;")
cursor.execute("GRANT CONNECT ON DATABASE crime_db TO readwrite;")

cursor.execute("GRANT USAGE ON SCHEMA crimes TO readonly;")
cursor.execute("GRANT USAGE ON SCHEMA crimes TO readwrite;")

cursor.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;")
cursor.execute("GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite;")
conn.commit()

Creating two imaginary users to check whether our user groups function as we designed them.

In [11]:
cursor.execute("DROP USER IF EXISTS data_analyst;")
cursor.execute("DROP USER IF EXISTS data_scientist;")

cursor.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")
cursor.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")
cursor.execute("GRANT readonly TO data_analyst;")
cursor.execute("GRANT readwrite TO data_scientist;")
conn.commit()
conn.close()

### Testing of the privileges
It is a good practice to check whether everything is set as expected, to format the input a little bit better we will use the pandas module and its read_sql function. This is not officially supported but it appears to be working for this purpose very well.

First let's take a look at our newly created users and user groups. By querying the pg_roles table we will be able to see what rights these groups/users have towards the database itself.

In [20]:
import pandas as pd
conn = psycopg2.connect(dbname="crime_db", user="postgres")
cursor = conn.cursor()
query = "SELECT * FROM pg_roles;"
pd.read_sql(query, conn)

/var/folders/jk/rfxssn693cd0q1t7mnndkd9h0000gn/T/ipykernel_86091/3758384077.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(query, conn)


,rolname,rolsuper,rolinherit,rolcreaterole,rolcreatedb,rolcanlogin,rolreplication,rolconnlimit,rolpassword,rolvaliduntil,rolbypassrls,rolconfig,oid
0,postgres,True,True,True,True,True,True,-1,********,None,True,None,10
1,pg_database_owner,False,True,False,False,False,False,-1,********,None,False,None,6171
2,pg_read_all_data,False,True,False,False,False,False,-1,********,None,False,None,6181
3,pg_write_all_data,False,True,False,False,False,False,-1,********,None,False,None,6182
4,pg_monitor,False,True,False,False,False,False,-1,********,None,False,None,3373
5,pg_read_all_settings,False,True,False,False,False,False,-1,********,None,False,None,3374
6,pg_read_all_stats,False,True,False,False,False,False,-1,********,None,False,None,3375
7,pg_stat_scan_tables,False,True,False,False,False,False,-1,********,None,False,None,3377
8,pg_read_server_files,False,True,False,False,False,False,-1,********,None,False,None,4569
9,pg_write_server_files,False,True,False,False,False,False,-1,********,None,False,None,4570


We can see both of our user groups and our two users in this table. We can also see that the user groups are indeed groups and not just two other users as they don't have login flag (rolcanlogin) set to True.

Now, let's check what actions can these user groups perform on the tables in our schema.

In [21]:
query = ("""
SELECT *
    FROM information_schema.table_privileges
    WHERE grantee = 'readonly';
    """)
pd.read_sql(query, conn)

/var/folders/jk/rfxssn693cd0q1t7mnndkd9h0000gn/T/ipykernel_86091/3557996346.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(query, conn)


,grantor,grantee,table_catalog,table_schema,table_name,privilege_type,is_grantable,with_hierarchy
0,postgres,readonly,crime_db,crimes,boston_crimes,SELECT,NO,YES


In [22]:
query = """
SELECT *
    FROM information_schema.table_privileges
    WHERE grantee = 'readwrite';
    """
pd.read_sql(query, conn)

/var/folders/jk/rfxssn693cd0q1t7mnndkd9h0000gn/T/ipykernel_86091/2812776399.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(query, conn)


,grantor,grantee,table_catalog,table_schema,table_name,privilege_type,is_grantable,with_hierarchy
0,postgres,readwrite,crime_db,crimes,boston_crimes,INSERT,NO,NO
1,postgres,readwrite,crime_db,crimes,boston_crimes,SELECT,NO,YES
2,postgres,readwrite,crime_db,crimes,boston_crimes,UPDATE,NO,NO
3,postgres,readwrite,crime_db,crimes,boston_crimes,DELETE,NO,NO


Everything appears to be set as we wanted. Further ways how to enhance this project could be to add a date table or a list of different offences if the city wanted to have more slicing capabilities and enrich the data for durther analysis. 